In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2024-10-09 14:05:28--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  12.7MB/s    in 0.4s    

2024-10-09 14:05:29 (12.7 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [25]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: ml-1m/movies.dat        
replace ml-1m/ratings.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [29]:
import pandas as pd
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='ISO-8859-1', names=['MovieID', 'Title', 'Genres'])

ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='ISO-8859-1', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])


,MovieID,Title,Genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama]",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),[Comedy],0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:

movies['Genres'] = movies['Genres'].str.split('|')
movies.head()
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres = pd.DataFrame(mlb.fit_transform(movies['Genres']), columns=mlb.classes_)
movies = pd.concat([movies, genres], axis=1)


,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(genres.values)


array([[1.        , 0.33333333, 0.40824829, ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40824829, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.70710678,
        1.        ]])

In [28]:
def recommend_movies(user_id, ratings, movies, similarity_matrix, top_n=10):
    user_ratings = ratings[ratings['UserID'] == user_id]
    user_movies = user_ratings['MovieID'].values
    recommended_movies = {}

    for movie_id in user_movies:
        movie_index = movies[movies['MovieID'] == movie_id].index[0]
        similar_movies = list(enumerate(similarity_matrix[movie_index]))
        similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

        for i, score in similar_movies[1:]:
            if movies.iloc[i]['MovieID'] not in user_movies:
                recommended_movies[movies.iloc[i]['Title']] = score

    recommended_movies = sorted(recommended_movies.items(), key=lambda x: x[1], reverse=True)
    return recommended_movies[:top_n]
recommendations = recommend_movies(1, ratings, movies, similarity_matrix)
for movie, score in recommendations:
    print(f'{movie}: {score}')


Othello (1995): 1.0
Now and Then (1995): 1.0
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995): 1.0
Dangerous Minds (1995): 1.0
Dead Man Walking (1995): 1.0
Cry, the Beloved Country (1995): 1.0
Restoration (1995): 1.0
Lamerica (1994): 1.0
Georgia (1995): 1.0
Home for the Holidays (1995): 1.0
